# Version control with git

## Introduction

This notebook illustrates some of the concepts when using the version control system git.  It is not intended as standalone training material, but rather as a practical illustration of the slides used in a training session.

To ensure that this notebook can be run from top to bottom, files are created using Bash here-documents, and edited using `sed`.  In practice, all editing would be done either with a GUI editor, or a command line editor such as `vim`.

You also want to make sure to start with a clean slate, so the project directory will be removed.

In [ ]:
readonly TOP_LEVEL=$(pwd) 2> /dev/null

In [ ]:
readonly PROJECT_DIR=my_project 2> /dev/null

In [ ]:
rm -rf "${PROJECT_DIR}"

## Global configuration

The first step is to set a few global settings, you have to do this only once. *Don't forget to modify the name and email address in the command below.*

~~~
git config --global user.name 'John Doe'
git config --global user.email 'john.doe@gmail.com'
~~~

Depending on your preferences, you may want to set an editor to use.  The default is `vim`.

~~~
git config --global core.editor nano
~~~

## Initialize repository

The git philosophy is to have a repository per project, so a new repository is initialized once for each new project.

In [ ]:
git init "${PROJECT_DIR}"
cd "${PROJECT_DIR}"

Apart from a directory `.git/`, the repository is empty.

In [ ]:
ls -a

Since the global settings were not changed, you can set the settings locally (so as not to mess up global values).

In [ ]:
git config user.name 'John Doe'
git config user.email 'john.doe@gmail.com'

## Edit/add/commit cycle

We can now add files, e.g.,

In [ ]:
cat > hello.c <<'EOI'
#include <stdio.h>

int main() {
    printf("hello world!\n");
}
EOI

Checking the status shows that `hello.c` is an untracked file, i.e., it is not under version control.  You can stage it, i.e., making it part of the next commit.

In [ ]:
git status

In [ ]:
git add hello.c

The status message now shows that `hello.c` is staged.

In [ ]:
git status

You can now commit the changes.  Note that you are supposed to provide an informative commit message (`-m` option).

In [ ]:
git commit -m 'Add hello world source code'

You can keep on working, e.g., adding a make file.

In [ ]:
cat > README.md <<"EOI"
Messenger
=========

Various utitlies to write messages to the terminal.
EOI

Add and commit the `README.md` file.

In [ ]:
git add README.md
git commit -m 'Add readme file'

## Log information

You can get an overview of commit by checking git's log.

In [ ]:
git log

There are many options to tune the log output, e.g., `--oneline`, which yields a very compact view.

In [ ]:
git log --oneline

## Ignoring files

However, not all files should be under version control, e.g., object files or executables.

In [ ]:
gcc -Wall -Wextra hello.c

Checking the status, you see that `a.out` is an untracked file, and you don't want it tracked, since it should not be in the repository.  It can be added to `.gitignore` so that git will ignore it in future.

In [ ]:
cat > .gitignore <<EOI
a.out
EOI

In [ ]:
git status

You should of course add `.gitignore` to the repository.

In [ ]:
git add .gitignore
git commit -m 'Ignore executable'

## Branching

Create a branch to develop a new feature.

In [ ]:
git branch feature/bye

List all the branches in the repository.  The one marked with `*` is the active branch.

In [ ]:
git branch

Switching to another branch is done through a check out.

In [ ]:
git checkout feature/bye
git branch

In [ ]:
cat > bye.c <<'EOI'
#include <stdio.h>

int main() {
    printf("bye world!\n");
    return 0;
}
EOI

Add and commit the changes.

In [ ]:
git add bye.c
git commit -m 'Add bye message'

## Merging

Meanwhile, a bug has been spotted in `hello.c`, this will be fixed in the `master` branch.  This is bad practice, but this way you'll see a merge.

In [ ]:
git checkout master

In [ ]:
sed -i "/printf/a\ \ \ \ return 0;" hello.c

Stage and commit `hello.c`.

In [ ]:
git commit -a -m 'Fix missing return statement'

In [ ]:
git log --oneline

Check out the feature branch to add a make file.

In [ ]:
git checkout feature/bye

In [ ]:
TAB=$'\t'
cat > Makefile <<EOI
CC = gcc
CFLAGS = -Wall -Wextra

all: bye.exe

%exe: %.o
${TAB}\$(CC) \$(CFLAGS) -o \$@ \$<
EOI

In [ ]:
git add Makefile
git commit -m 'Add make file for bye application'

To ensure you get a conflict for illustration purposes, check out the master branch, and create a Makefile for `hello.exe` as well.

In [ ]:
git checkout master

Note that in this branch, `Makefile` doesn't exist.

In [ ]:
ls

Add and commit the make file for `hello.exe`.

In [ ]:
TAB=$'\t'
cat > Makefile <<EOI
CC = gcc
CFLAGS = -Wall -Wextra

all: hello.exe

%exe: %.o
${TAB}\$(CC) \$(CFLAGS) -o \$@ \$<
EOI

In [ ]:
git add Makefile
git commit -m 'Add make file for hello.exe'

Now you can merge the changes made in `feature/bye` into the `master` branch.  There will be a conflict on the make file.

In [ ]:
git merge feature/bye

The  conflict should be resolved.

In [ ]:
cat Makefile

You should fix the conflict indicated by

~~~
<<<<<<< HEAD
all: hello.exe
=======
all: bye.exe
>>>>>>> feature/bye
~~~

These five lines should be replaced by what you actually want, i.e.,

~~~
all: hello.exe bye.exe
~~~

In [ ]:
sed -i '4,8d; 3aall: hello.exe bye.exe' Makefile

The conflict is resolved, the make file can be added and committed.

In [ ]:
git status

In [ ]:
git add Makefile
git commit -m 'Add bye application'

Inspectiing the history shows that all commits are now in `master`, in chronological order.

In [ ]:
git log

The feature branch can now be deleted.

In [ ]:
git branch -d feature/bye

## Squashing commits

Having all the commit messages merged may be confusing, and lead to a history that is hard to understand. You can squash all commits in a branch into one prior to merging it back in. To illustrate this, you can create a new branch with several commits.

In [ ]:
git checkout -b feature/cli_arg

Modify `hello.c` so that it takes a command line argument, stage and commit.

In [ ]:
sed -e 's/main()/main(int argc, char *argv[])/' \
    -e 's/world!\\n"/%s!\\n", argv[1]/' -i hello.c

In [ ]:
git commit -a -m 'Add command line argument to hello'

Do the same for `bye.c`.

In [ ]:
sed -e 's/main()/main(int argc, char *argv[])/' \
    -e 's/world!\\n"/%s!\\n", argv[1]/' -i bye.c

In [ ]:
git commit -a -m 'Add command line argument to bye'

*Note:* it is very bad style to use a command line argument value without checking it was given on the command line.

Add a clean rule to the make file.

In [ ]:
CLEAN='clean:\n\t$(RM) $(wildcard *.exe)'
sed "\$a'${CLEAN}'" -i Makefile

Stage and commit the make file.

In [ ]:
git commit -a -m 'Add clean rule'

Meanwhile in the `master` branch, `README.md` is updated.

In [ ]:
git checkout master

In [ ]:
CONTENT='\nContent\n-------\n\n- `hello.exe\n- `bye.exe`'
sed "\$a'${CONTENT}''" -i README.md

In [ ]:
git commit -a -m 'Add content'

Merging now would insert the three commits done in `feature/cli_arg` into `master`'s history, before this last commit. If that is not what you want, you can squash the commits in `feature/cli_arg` before merging.

In [ ]:
git checkout feature/cli_arg

When checking the commits in the history of this branch, you see that the current branch has three commits, which you may want to squash into one.

In [ ]:
git log --oneline -5

In [ ]:
git reset --soft HEAD~3
git commit -m 'Implement command line arguments'

*Note:* this rewrites history, i.e., you loose (potentially valuable) information.  It depends on policies whether that is acceptable or not.

Inspecting the history, you see that the three last commits have been replaced by a single one.

In [ ]:
git log --oneline -4

Now this commit can be included in the `master` branch.  First, switch back to the `master` branch.

In [ ]:
SQUASH_COMMIT=$(git log --oneline -1 | cut -d' ' -f1)

In [ ]:
git checkout master

You could use `git merge feature/cli_arg` to merge the changes made in that branch into `master`.  However, that would result in a merge commit.  If you want to avoid that, you can simply cherry-pick the squashed commit from `feature/cli_arg`.

In [ ]:
git cherry-pick "${SQUASH_COMMIT}"

When you inspect the log, you see that the last commit seems to have been done directly into `master`, there is no trace of a merge.

In [ ]:
git log --oneline --graph

*Note:* again, it depends on policies whether this is deemed acceptable.

## Clean up

Move out of the directory, and remove it to delete the repository.

In [ ]:
cd "${TOP_LEVEL}"
rm -rf "${PROJECT_DIR}"